# Setup

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.8 MB/s eta 0:00:00


In [ ]:
from scipy.spatial.distance import cosine

### Utils
bert-base-uncased and bert-base-multilingual-cased have been evaluated

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from transformers import AutoTokenizer, BertModel
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import AutoTokenizer, BertModel
model = BertModel.from_pretrained('bert-base-multilingual-cased', output_hidden_states = True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from Utils import BERT_Embeddings, preprocessing, tokens_evaluation
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd
import numpy as np
import torch
from IPython.display import HTML
from tqdm import tqdm

### Load training Data

In [ ]:
train_df = pd.read_json("./MD-Agreement_dataset/MD-Agreement_train.json", orient='index')
train_df = preprocessing.get_dataset_labels(train_df)

train_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement
1,<user> <user> No way Jose!!,0,1.0,0.0,1
2,"Good god, what is the matter with people ?",0,1.0,0.0,1
3,<user> <user> <user> <user> Um the Kurds are h...,0,0.8,0.2,0
4,What is WRONG with these people?,0,1.0,0.0,1
5,<user> This earpiece too plus a wire on his sl...,0,0.8,0.2,0
...,...,...,...,...,...
6588,Instead of showing compassion and empathy..thi...,0,1.0,0.0,1
6589,<user> Neither did you,0,1.0,0.0,1
6590,"""University of Toronto, McMaster University, S...",0,1.0,0.0,1
6591,The only people ripping this country apart are...,0,0.6,0.4,0


In [ ]:
# sentences
train_df['sentences'] = train_df['original_text'].apply(lambda x : preprocessing.split_sentence(x))
train_df['sentences'] = train_df['sentences'].apply(lambda x :preprocessing.adjust_split(x))
train_df['sentences'] = train_df['sentences'].apply(lambda x : preprocessing.apply_lemmatization(x))


In [ ]:
train_df['tokens_lists'] = train_df['sentences'].apply(lambda x: preprocessing.sentences_tokenizer(x))
train_df['tokens_lists']= train_df['tokens_lists'].apply(lambda x: preprocessing.clear_tokens(x))
train_df['tokens_list']= train_df['tokens_lists'].apply(lambda x: preprocessing.flatten_list(x))

In [ ]:
train_df

In [ ]:
train_df['original_text']= train_df['tokens_list'].apply(lambda x: ' '.join(x))
train_df['original_text'] = train_df['original_text'].apply(lambda x :x.replace('\u200d', ''))
train_df

### Load Dev Data

In [ ]:
dev_df = pd.read_json("./MD-Agreement_dataset/MD-Agreement_dev.json", orient='index')
dev_df = preprocessing.get_dataset_labels(dev_df)

In [ ]:
# sentences
dev_df['sentences'] = dev_df['original_text'].apply(lambda x : preprocessing.split_sentence(x))
dev_df['sentences'] = dev_df['sentences'].apply(lambda x :preprocessing.adjust_split(x))
dev_df['sentences'] = dev_df['sentences'].apply(lambda x : preprocessing.apply_lemmatization(x))
dev_df['tokens_lists'] = dev_df['sentences'].apply(lambda x: preprocessing.sentences_tokenizer(x))
dev_df['tokens_lists']= dev_df['tokens_lists'].apply(lambda x: preprocessing.clear_tokens(x))
dev_df['tokens_list']= dev_df['tokens_lists'].apply(lambda x: preprocessing.flatten_list(x))

In [ ]:
dev_df['original_text']= dev_df['tokens_list'].apply(lambda x: ' '.join(x))
dev_df['original_text'] = dev_df['original_text'].apply(lambda x :x.replace('\u200d', ''))
dev_df

### Load Test Data

In [ ]:
test_df = pd.read_json("./MD-Agreement_dataset/MD-Agreement_test.json", orient='index')
test_df = preprocessing.get_dataset_labels(test_df)

In [ ]:
# sentences
test_df['sentences'] = test_df['original_text'].apply(lambda x : preprocessing.split_sentence(x))
test_df['sentences'] = test_df['sentences'].apply(lambda x :preprocessing.adjust_split(x))
test_df['sentences'] = test_df['sentences'].apply(lambda x : preprocessing.apply_lemmatization(x))
test_df['tokens_lists'] = test_df['sentences'].apply(lambda x: preprocessing.sentences_tokenizer(x))
test_df['tokens_lists']= test_df['tokens_lists'].apply(lambda x: preprocessing.clear_tokens(x))
test_df['tokens_list']= test_df['tokens_lists'].apply(lambda x: preprocessing.flatten_list(x))

In [ ]:
test_df['original_text']= test_df['tokens_list'].apply(lambda x: ' '.join(x))
test_df['original_text'] = test_df['original_text'].apply(lambda x :x.replace('\u200d', ''))
test_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement,sentences,tokens_lists,tokens_list
1,million unemployed fool,0,0.8,0.2,0,"[<user> <user> 30 million unemployed, fool]","[[million, unemployed], [fool]]","[million, unemployed, fool]"
2,#wedontbuyhate powerful watch,0,1.0,0.0,1,[<user> <user> <user> <user> <user> <user> #We...,"[[#wedontbuyhate, powerful], [watch]]","[#wedontbuyhate, powerful, watch]"
3,watch entire #georgefloyd porno unlock inner l...,1,0.0,1.0,1,"[if you watch the entire #GeorgeFloyd porno, y...","[[watch, entire, #georgefloyd, porno], [unlock...","[watch, entire, #georgefloyd, porno, unlock, i..."
4,stay away child,0,0.6,0.4,0,[<user> stay away from child],"[[stay, away, child]]","[stay, away, child]"
5,super spreader accusation people party street ...,1,0.2,0.8,0,"[<user> <user> no super, spreader accusation a...","[[super], [spreader, accusation, people, party...","[super, spreader, accusation, people, party, s..."
...,...,...,...,...,...,...,...,...
3053,bold faced liar uncover,1,0.4,0.6,0,"[<user> you ’re a bold faced liar, you will be...","[[bold, faced, liar], [uncover]]","[bold, faced, liar, uncover]"
3054,hey nj even vote person murphy fn disgrace,0,0.6,0.4,0,[<user> <user> <user> hey NJ even <user> can v...,"[[hey, nj, even, vote, person], [murphy, fn, d...","[hey, nj, even, vote, person, murphy, fn, disg..."
3055,people ask peer hate city 🤬 literally ‼ ai 🙅 ...,0,0.6,0.4,0,[and people ask I and my peer why we hate our ...,"[[people, ask, peer, hate, city], [🤬, literall...","[people, ask, peer, hate, city, 🤬, literally, ..."
3056,love black people much george floyd service to...,0,0.8,0.2,0,[<user> if you love we BLACK People so much ...,"[[love, black, people, much, george, floyd, se...","[love, black, people, much, george, floyd, ser..."


# Baseline

In [ ]:
tokens_df = pd.read_csv('./Nuovi_Scores/MD-Agreement_scores.csv', sep='\t')
tokens_df_10 = tokens_df[tokens_df.occurrences >= 10]

### Threshold estimation from Dev

In [ ]:
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []

for _, row in tqdm(dev_df.iterrows()):
  colors_agreement, _ = tokens_evaluation.get_all_colors(row['tokens_list'], tokens_df_10)

  if 'NA' in colors_agreement:
    indexes = tokens_evaluation.find_NA_indices(colors_agreement)
    for new_word_index in range(len(indexes)) :
      colors_agreement[indexes[new_word_index]]=0

  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

1104it [00:06, 158.49it/s]


In [ ]:
best_t = 0
best_f1 = 0
for t in np.arange(round(min(pred_somma)), round(max(pred_somma)), 0.1):
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred_somma], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('SOMMA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred_somma] ))

SOMMA 

THRESHOLD: -1.400000000000027

              precision    recall  f1-score   support

           0       0.67      0.74      0.71       639
           1       0.59      0.51      0.54       465

    accuracy                           0.64      1104
   macro avg       0.63      0.63      0.63      1104
weighted avg       0.64      0.64      0.64      1104



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_media
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('MEDIA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

MEDIA 

THRESHOLD: -0.3

              precision    recall  f1-score   support

           0       0.66      0.54      0.59       639
           1       0.49      0.61      0.54       465

    accuracy                           0.57      1104
   macro avg       0.57      0.58      0.57      1104
weighted avg       0.59      0.57      0.57      1104



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_mediana
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('MEDIANA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

MEDIANA 

THRESHOLD: -0.3

              precision    recall  f1-score   support

           0       0.66      0.60      0.63       639
           1       0.51      0.58      0.55       465

    accuracy                           0.59      1104
   macro avg       0.59      0.59      0.59      1104
weighted avg       0.60      0.59      0.59      1104



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_tutti_verdi
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print(' ALL GREEN \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

 ALL GREEN 

THRESHOLD: -0.5

              precision    recall  f1-score   support

           0       0.68      0.62      0.65       639
           1       0.54      0.60      0.57       465

    accuracy                           0.61      1104
   macro avg       0.61      0.61      0.61      1104
weighted avg       0.62      0.61      0.61      1104



### Performances on Test

In [ ]:
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []

for _, row in tqdm(test_df.iterrows()):
  colors_agreement, _ = tokens_evaluation.get_all_colors(row['tokens_list'], tokens_df_10)

  if 'NA' in colors_agreement:
    indexes = tokens_evaluation.find_NA_indices(colors_agreement)
    for new_word_index in range(len(indexes)) :
       colors_agreement[indexes[new_word_index]]=0

  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

3057it [00:19, 153.40it/s]


In [ ]:
print('SOMMA \n')
print(classification_report(test_df['disagreement'], [int(i>=-1.4) for i in pred_somma] ))

print('\n MEDIA \n')
print(classification_report(test_df['disagreement'], [int(i>=-0.3) for i in pred_media]))

print('\n MEDIANA \n')
print(classification_report(test_df['disagreement'], [int(i>=-0.3) for i in pred_mediana]))

print('\n ALL GREEN \n')
print(classification_report(test_df['disagreement'], [int(i>=-0.5) for i in pred_tutti_verdi]))

stima NA: NO
soglia coordinate: almeno 10 occorrenze
valori non pesati
SOMMA 

              precision    recall  f1-score   support

           0       0.64      0.69      0.67      1765
           1       0.53      0.47      0.50      1292

    accuracy                           0.60      3057
   macro avg       0.59      0.58      0.58      3057
weighted avg       0.59      0.60      0.60      3057


 MEDIA 

              precision    recall  f1-score   support

           0       0.64      0.53      0.58      1765
           1       0.48      0.59      0.53      1292

    accuracy                           0.55      3057
   macro avg       0.56      0.56      0.55      3057
weighted avg       0.57      0.55      0.56      3057


 MEDIANA 

              precision    recall  f1-score   support

           0       0.63      0.56      0.60      1765
           1       0.48      0.56      0.52      1292

    accuracy                           0.56      3057
   macro avg       0.56    

In [ ]:
# open file in write mode
with open(r'./Results_NoEstimation/MD_Sum.txt', 'w') as fp:
    for item in pred_somma:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'./Results_NoEstimation/MD_Mean.txt', 'w') as fp:
    for item in pred_media:
        # write each item on a new line
        fp.write("%s\n" % item)


with open(r'./Results_NoEstimation/MD_Median.txt', 'w') as fp:
    for item in pred_mediana:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'./Results_NoEstimation/MD_Min.txt', 'w') as fp:
    for item in pred_tutti_verdi:
        # write each item on a new line
        fp.write("%s\n" % item)

# Mean contextualized
mean of the contextualized vectors to obtain one vector per token

## Context Embeddings for token in Train

In [ ]:
sentences = list(train_df['original_text'])

In [ ]:
from collections import OrderedDict

context_embeddings = []
context_tokens = []

for sentence in sentences:
  while sentence and sentence[len(sentence)-1] == '*' :
    sentence = sentence[:len(sentence)-1]
  tokenized_text, list_token_embeddings = BERT_Embeddings.text_to_emb(sentence, tokenizer, model)

  # make ordered dictionary to keep track of the position of each word
  tokens = OrderedDict()

  # loop over tokens in sensitive sentence
  for token in tokenized_text[1:-1]:
    # keep track of position of word and whether it occurs multiple times
    if token in tokens:
      tokens[token] += 1
    else:
      tokens[token] = 1

    # compute the position of the current token
    token_indices = [i for i, t in enumerate(tokenized_text) if t == token]
    current_index = token_indices[tokens[token]-1]

    # get the corresponding embedding
    token_vec = list_token_embeddings[current_index]

    # save values
    context_tokens.append(token)
    context_embeddings.append(token_vec)

In [ ]:
def find_token_indices(list_to_check, token):
    """
    Finds the indices of 'NA' values in a list.

    Args:
        list_to_check (list): List to check for 'NA' values.

    Returns:
        list: List of indices where 'NA' values are found.

    """
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == token:
            indices.append(idx)
    return indices

In [ ]:
def retrieve_elements(lst, indexes):
    return [lst[i] for i in indexes]

In [ ]:
context_embeddings_mean = []
context_tokens_mean = []

for tk in set(context_tokens):
  indexes = find_token_indices(context_tokens, tk)
  context_tokens_mean.append(tk)
  context_embeddings_mean.append(torch.mean(torch.stack(retrieve_elements(context_embeddings, indexes)) , dim=0))

print(len(context_tokens_mean))

10376


In [ ]:
#tengo solo quelli con 10 occ
context_embeddings_mean_2 = []
context_tokens_mean_2 = []

for tk in set(context_tokens_mean):
  if tk in list(tokens_df_10['token']):
    indexes = find_token_indices(context_tokens_mean, tk)
    context_tokens_mean_2.append(tk)
    context_embeddings_mean_2.append(context_embeddings_mean[indexes[0]])

context_embeddings_mean = context_embeddings_mean_2
context_tokens_mean = context_tokens_mean_2

### G-Models

#### Threshold on DEV

In [ ]:
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []
threshold=0.6

for _, row in tqdm(dev_df.iterrows()):
  colors_agreement, _ = tokens_evaluation.get_all_colors(row['original_text'].split(), tokens_df_10)

  if 'NA' in colors_agreement:
    similar_words, distances_df, new_words  = find_similar_words(row['original_text'], tokens_evaluation.find_NA_indices(colors_agreement), tokenizer, context_tokens_mean,context_embeddings_mean, model, tokens_df_10)
    for i in range(0, len(new_words)):
      if list(distances_df.loc[(distances_df['new_token']==new_words[i])& (distances_df['distance']>=threshold)]['distance']):
        stimated_coordinate = np.average(
            list(distances_df.loc[(distances_df['new_token']==new_words[i]) & (distances_df['distance']>=threshold)]['Agreement_coordinate']),
            weights=list(distances_df.loc[(distances_df['new_token']==new_words[i])& (distances_df['distance']>=threshold)]['distance']))

        colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=stimated_coordinate
      else: #if there isn't any word above the threshold
        colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=0
  while 'NA' in colors_agreement:
    colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=0
  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

131it [08:25,  1.72s/it]

['tribe', 'defend', 'pandemic', 'south', 'dakota', 'state', 'government', '#shameonyou']
['tribe', 'defend', 'pandemic', 'south', 'dakota', 'state', 'government', '#shameonyou', '\u2066']


191it [11:45,  1.56s/it]

['amount', 'man', 'see', 'flat', 'refuse', 'wash', 'hand', 'use', 'bathroom', 'appalling', 'surprise', 'still', 'disgust', 'also', 'amount', 'dont', 'fucking', 'flush', 'breh', 'lever', 'right', 'fucking*internally', 'fuck', 'yelling*<', 'url', '>']
['amount', 'man', 'see', 'flat', 'refuse', 'wash', 'hand', 'use', 'bathroom', 'appalling', 'surprise', 'still', 'disgust', 'also', 'amount', 'dont', 'fucking', 'flush', 'breh', 'lever', 'right', 'fucking*internally', 'fuck', 'yelling*<url>']


276it [16:10,  2.42s/it]

...

835it [50:15,  3.62s/it]

['kemp', 'claim', 'find', 'hour', 'ago', 'symptomatic', 'people', 'could', 'spread', 'covid', 'call', 'bullsh*stupidly', 'try', 'cover', 'hang', 'everyone', 'see']
['kemp', 'claim', 'find', 'hour', 'ago', 'symptomatic', 'people', 'could', 'spread', 'covid', 'call', 'bullsh*', 'stupidly', 'try', 'cover', 'hang', 'everyone', 'see']


931it [55:43,  2.62s/it]

['#wehoprotest', 'timelapse', 'amp', 'gay', 'agenda', 'vignette', '#armthegays', '🏳', '🌈', '✊', '🏿', '#blm', '#timelapse', '#lapride', '#laprotest', '#westhollywood', '#ablmla', '#blacklivesmatter', '#pride']
['#wehoprotest', 'timelapse', 'amp', 'gay', 'agenda', 'vignette', '#armthegays', '🏳', '️', '🌈', '✊', '🏿', '#blm', '#timelapse', '#lapride', '#laprotest', '#westhollywood', '#ablmla', '#blacklivesmatter', '#pride']


1104it [1:05:39,  3.57s/it]


In [ ]:
best_t = 0
best_f1 = 0
for t in np.arange(round(min(pred_somma)), round(max(pred_somma)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred_somma], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('SOMMA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred_somma] ))

SOMMA 

THRESHOLD: -1.7

              precision    recall  f1-score   support

           0       0.66      0.80      0.72       639
           1       0.61      0.44      0.52       465

    accuracy                           0.65      1104
   macro avg       0.64      0.62      0.62      1104
weighted avg       0.64      0.65      0.64      1104



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_media
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('MEDIA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

MEDIA 

THRESHOLD: -0.3

              precision    recall  f1-score   support

           0       0.65      0.53      0.58       639
           1       0.48      0.61      0.54       465

    accuracy                           0.56      1104
   macro avg       0.57      0.57      0.56      1104
weighted avg       0.58      0.56      0.56      1104



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_mediana
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('MEDIANA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

MEDIANA 

THRESHOLD: -0.3

              precision    recall  f1-score   support

           0       0.62      0.47      0.53       639
           1       0.46      0.62      0.52       465

    accuracy                           0.53      1104
   macro avg       0.54      0.54      0.53      1104
weighted avg       0.55      0.53      0.53      1104



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_tutti_verdi
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print(' ALL GREEN \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

 ALL GREEN 

THRESHOLD: -0.5

              precision    recall  f1-score   support

           0       0.68      0.63      0.65       639
           1       0.54      0.58      0.56       465

    accuracy                           0.61      1104
   macro avg       0.61      0.61      0.61      1104
weighted avg       0.62      0.61      0.61      1104



#### Performances on Test

In [ ]:
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []
threshold=0.6

for _, row in tqdm(test_df.iterrows()):
  colors_agreement, _ = tokens_evaluation.get_all_colors(row['original_text'].split(), tokens_df_10)

  if 'NA' in colors_agreement:
    similar_words, distances_df, new_words  = find_similar_words(row['original_text'], tokens_evaluation.find_NA_indices(colors_agreement), tokenizer, context_tokens_mean,context_embeddings_mean, model, tokens_df_10)
    for i in range(0, len(new_words)):
      if list(distances_df.loc[(distances_df['new_token']==new_words[i])& (distances_df['distance']>=threshold)]['distance']):
        stimated_coordinate = np.average(
            list(distances_df.loc[(distances_df['new_token']==new_words[i]) & (distances_df['distance']>=threshold)]['Agreement_coordinate']),
            weights=list(distances_df.loc[(distances_df['new_token']==new_words[i])& (distances_df['distance']>=threshold)]['distance']))

        colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=stimated_coordinate
      else: #if there isn't any word above the threshold
        colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=0
  while 'NA' in colors_agreement:
    colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=0

  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

40it [02:55,  2.88s/it]

['chinese', 'overlord', '操', '你']
['chinese', 'overlord', '操你']


104it [06:51,  2.30s/it]

['yep', 'cop*pissed*👍']
['yep', 'cop*pissed*', '👍']


107it [06:56,  1.75s/it]

...

2768it [2:51:13,  3.21s/it]

['murder', 'george', 'floyd****<', 'url', '>']
['murder', 'george', 'floyd****<url>']


2837it [2:54:50,  2.71s/it]

['good', 'news', 'screw', 'even', 'win', '#senatemajority']
['good', 'news', '\u2066', 'screw', 'even', 'win', '#senatemajority']


3057it [3:08:56,  3.71s/it]


In [ ]:
print('SOMMA \n')
print(classification_report(test_df['disagreement'], [int(i>=-1.7) for i in pred_somma] ))

print('\n MEDIA \n')
print(classification_report(test_df['disagreement'], [int(i>=-0.3) for i in pred_media]))

print('\n MEDIANA \n')
print(classification_report(test_df['disagreement'], [int(i>=-0.3) for i in pred_mediana]))

print('\n ALL GREEN \n')
print(classification_report(test_df['disagreement'], [int(i>=-0.5) for i in pred_tutti_verdi]))

stima NA: NO
soglia coordinate: almeno 10 occorrenze
valori non pesati
SOMMA 

              precision    recall  f1-score   support

           0       0.64      0.76      0.69      1765
           1       0.56      0.41      0.47      1292

    accuracy                           0.61      3057
   macro avg       0.60      0.59      0.58      3057
weighted avg       0.60      0.61      0.60      3057


 MEDIA 

              precision    recall  f1-score   support

           0       0.64      0.52      0.57      1765
           1       0.48      0.61      0.54      1292

    accuracy                           0.56      3057
   macro avg       0.56      0.56      0.56      3057
weighted avg       0.57      0.56      0.56      3057


 MEDIANA 

              precision    recall  f1-score   support

           0       0.63      0.48      0.54      1765
           1       0.46      0.62      0.53      1292

    accuracy                           0.54      3057
   macro avg       0.55    

In [ ]:
# open file in write mode
with open(r'./MD_G-Sum.txt', 'w') as fp:
    for item in pred_somma:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'./MD_G-Mean.txt', 'w') as fp:
    for item in pred_media:
        # write each item on a new line
        fp.write("%s\n" % item)


with open(r'./MD_G-Median.txt', 'w') as fp:
    for item in pred_mediana:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'./MD_G-Min.txt', 'w') as fp:
    for item in pred_tutti_verdi:
        # write each item on a new line
        fp.write("%s\n" % item)